<a href="https://colab.research.google.com/github/sbuttler/ddnv/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn==0.23.1 tensorflow==2.1.0  keras==2.3.1 ddop==0.6.7

In [1]:
from sklearn.utils.validation import check_array
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from ddop.datasets import load_yaz, load_bakery
from ddop.metrics import average_costs, prescriptiveness_score
from ddop.newsvendor import SampleAverageApproximationNewsvendor
from ddop.newsvendor import DecisionTreeWeightedNewsvendor
from ddop.newsvendor import RandomForestWeightedNewsvendor 
from ddop.newsvendor import KNeighborsWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import DeepLearningNewsvendor
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import statistics

In [2]:
n_features = 12

In [18]:
np.arange(1,11)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [27]:
#TODO: Set Grids
dl = {"optimizer": ["adam"],
      "neurons": [
                  (round(0.5*n_features),round(0.5*0.5*n_features)),
                  (round(0.5*n_features),round(0.5*1*n_features)),
                  (1*n_features,round(1*0.5*n_features)),
                  (1*n_features,1*1*n_features),
                  (2*n_features,round(2*0.5*n_features)),
                  (2*n_features,2*1*n_features),
                  (3*n_features,round(3*0.5*n_features)),
                  (3*n_features,3*1*n_features)],
      "epochs": [10,100,200]}

dtw = {"max_depth":[1,2,3,4,5,6,7,8,9,10,15]}

rfw = {"max_depth":[None,2,4,6,8,10],
          'min_samples_split':[2,4,6,8,10],
          'n_estimators':[2,10,20]}

knnw = {'n_neighbors':[1,2,4,8,16,32, 64,128]}

gkw = {'kernel_bandwidth':[1,1.2,1.4,1.6,1.8,2,2.5,3]}

In [28]:
# Define model tuples: 'model_name', model, grid
estimator_tuple_list = []
#estimator_tuple_list.append(('SAA', SampleAverageApproximationNewsvendor(),None))
#estimator_tuple_list.append(('DTW', DecisionTreeWeightedNewsvendor(random_state=1),dtw))
#estimator_tuple_list.append(('RFW', RandomForestWeightedNewsvendor(random_state=1),rfw))
estimator_tuple_list.append(('KNNW',KNeighborsWeightedNewsvendor(),knnw))
#estimator_tuple_list.append(('GKW', GaussianWeightedNewsvendor(),gkw))
#estimator_tuple_list.append(('DL', DeepLearningNewsvendor(),[dl]))
#estimator_tuple_list.append(('LR', LinearRegressionNewsvendor(),None))

--------------------------------------
# YAZ Dataset

In [29]:
data = load_yaz(one_hot_encoding=True)
X = data.data
y = data.target

In [30]:
products = y.columns.to_list()

No target scaling

In [45]:
estimators = []
results = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for product in products:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y[product], train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
        
      else:
        gs = GridSearchCV(estimator, param_grid)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[products[i]+" AC"] = costs[i]
    for i in range(len(costs)):
      d[products[i]+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])

DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=5, cu=5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=2.5, cu=7.5, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvendor(co=1, cu=9, random_state=1)
DecisionTreeWeightedNewsvend

In [ ]:
results

In [ ]:
estimators

With target scaling and 10 fold CV

In [31]:
estimators = []
results_sc = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for product in products:
      
      X_train, X_test, y_train, y_test = train_test_split(X, y[product], train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)
      
      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))
      #y_test = scaler_target.transform(np.array(y_test).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv, n_jobs=-1)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[products[i]+" AC"] = costs[i]
    for i in range(len(costs)):
      d[products[i]+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results_sc = pd.concat([results_sc,df])

KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=2.5, cu=7.5)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)
KNeighborsWeightedNewsvendor(co=1, cu=9)


In [32]:
results_sc

,SL,Model,calamari AC,fish AC,shrimp AC,chicken AC,koefte AC,lamb AC,steak AC,calamari SoP,fish SoP,shrimp SoP,chicken SoP,koefte SoP,lamb SoP,steak SoP,Average Cost,Score of Prescriptiveness
0,0.50,KNNW,8.203125,8.776042,16.510417,48.046875,36.119792,54.062500,29.427083,0.073529,0.017493,0.062130,-0.125000,0.044108,-0.138158,0.079805,28.735119,0.001987
0,0.75,KNNW,7.486979,7.773438,13.333333,30.833333,29.934896,32.786458,23.190104,0.078526,0.016474,0.112652,0.192911,0.068099,0.185640,0.226325,20.762649,0.125804
0,0.90,KNNW,4.520833,5.192708,7.406250,20.567708,15.765625,21.197917,14.125000,0.126761,0.094460,0.129743,0.167580,0.164966,0.031414,0.279873,12.682292,0.142114


In [33]:
estimators

[KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=2.5, cu=7.5, n_neighbors=32),
 KNeighborsWeightedNewsvendor(co=1, cu=9, n_neighbors=64),
 KNeighborsWeightedNewsvendor(co=1, cu=9, n_neighbors=128),
 KNeighborsWeightedNewsvendor(c

In [ ]:
results.to_excel("results_yaz_lag_per_product.xlsx")

In [ ]:
from google.colab import files

In [ ]:
files.download('results_yaz_lag_per_product.xlsx') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-----------------------------------------------------------------
# Global Model (ausgegliedert)

In [ ]:
estimators = []
results = pd.DataFrame()
for cu, co in zip([5, 7.5, 9],[5, 2.5, 1]):
  saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
  for estimator_tuple in estimator_tuple_list:
    estimator_name = estimator_tuple[0]
    param_grid = estimator_tuple[2]
    estimator = estimator_tuple[1]
    estimator.set_params(cu=cu,co=co)
    print(estimator)
    if param_grid == None:
      if estimator_name=="SAA":
        pred = estimator.fit(y_train).predict(X_test.shape[0])
      else:
        pred = estimator.fit(X_train,y_train).predict(X_test)
      
    else:
      gs = GridSearchCV(estimator, param_grid)
      gs.fit(X_train,y_train)
      best_estimator = gs.best_estimator_
      pred = best_estimator.predict(X_test)
      estimators.append(best_estimator)
      
    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="raw_values")
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="raw_values")
    for i in range(avg_cost.size):
      d[products[i]+" AC"] = avg_cost[i]
    for i in range(avg_cost.size):
      d[products[i]+" SoP"] = p_score[i]
    avg_cost = average_costs(y_test,pred,cu,co, multioutput="uniform_average")
    d["Average Cost"] = avg_cost
    p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
    d["Score of Prescriptiveness"] = p_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])


-----------------------------------------
# Bakery Dataset

In [34]:
bakery = load_bakery(one_hot_encoding=True)
X = bakery.data
y = bakery.target

In [35]:
X_grouped = X.groupby(['product', 'store'])

In [36]:
groups = list(X_grouped.groups.keys())

In [37]:
import statistics
estimators = []
results = pd.DataFrame()
for cu, co in zip([5,7.5,9],[5,2.5,1]):
#for cu, co in zip([7.5],[2.5]):
  for  estimator_tuple in estimator_tuple_list:
    costs = []
    score = []
    for group in groups:

      X_temp = X_grouped.get_group(group)
      y_temp = y.iloc[X_temp.index.values.tolist()]

      X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, train_size=0.75, shuffle=False)
      scaler = StandardScaler()
      scaler.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)

      #scale target variable
      scaler_target = StandardScaler()
      scaler_target.fit(np.array(y_train).reshape(-1, 1))
      y_train = scaler_target.transform(np.array(y_train).reshape(-1, 1))

      saa_pred = SampleAverageApproximationNewsvendor(cu,co).fit(y_train).predict(y_test.shape[0])
      saa_pred = scaler_target.inverse_transform(saa_pred)
      estimator_name = estimator_tuple[0]
      param_grid = estimator_tuple[2]
      estimator = estimator_tuple[1]
      estimator.set_params(cu=cu,co=co)
      print(estimator)

      if param_grid == None:
        if estimator_name=="SAA":
          pred = estimator.fit(y_train).predict(X_test.shape[0])
          pred = scaler_target.inverse_transform(pred)
        else:
          pred = estimator.fit(X_train,y_train).predict(X_test)
          pred = scaler_target.inverse_transform(pred)
        
      else:
        cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
        gs = GridSearchCV(estimator, param_grid, cv=cv, n_jobs=-1)
        gs.fit(X_train,y_train)
        best_estimator = gs.best_estimator_
        pred = best_estimator.predict(X_test)
        pred = scaler_target.inverse_transform(pred)
        estimators.append(best_estimator)

      avg_cost = average_costs(y_test,pred,cu,co,multioutput="uniform_average")
      p_score = prescriptiveness_score(y_test, pred, saa_pred, cu, co, multioutput="uniform_average")
      costs.append(avg_cost)
      score.append(p_score)

    d = {'SL': [cu/(cu+co)], 'Model': [estimator_name]}  
    for i in range(len(costs)):
      d[str(groups[i])+" AC"] = costs[i]
    for i in range(len(costs)):
      d[str(groups[i])+" SoP"] = score[i]

    average_cost = statistics.mean(costs)
    d["Average Cost"] = average_cost
    presc_score = statistics.mean(score)
    d["Score of Prescriptiveness"] = presc_score
    df = pd.DataFrame(data=d)
    results = pd.concat([results,df])

KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeightedNewsvendor(co=5, cu=5)
KNeighborsWeight

In [38]:
results

,SL,Model,"(101, 2) AC","(101, 3) AC","(101, 4) AC","(101, 5) AC","(101, 17) AC","(101, 19) AC","(101, 20) AC","(101, 21) AC",...,"(110, 45) SoP","(110, 46) SoP","(110, 47) SoP","(110, 48) SoP","(110, 49) SoP","(110, 50) SoP","(110, 70) SoP","(110, 71) SoP",Average Cost,Score of Prescriptiveness
0,0.50,KNNW,180.328947,150.904605,12.187500,684.128125,202.023026,399.111842,206.266447,290.444079,...,0.079065,0.500000,0.682794,0.151951,0.339788,0.379152,0.174388,0.028433,156.195138,0.225580
0,0.75,KNNW,153.281250,133.758224,12.228618,1025.279359,191.422697,344.950658,178.791118,248.379934,...,0.079324,0.453099,0.670923,0.021827,0.333128,0.351195,0.209583,0.004665,137.243537,0.254918
0,0.90,KNNW,122.648026,70.243421,8.779605,1209.729967,148.562500,217.898026,154.677632,157.901316,...,0.005263,0.358560,0.582358,0.011348,0.112330,0.248646,0.201206,0.032218,97.350976,0.248497


In [39]:
estimators

[KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=8),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=128),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=32),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=2),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=8),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=8),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=8),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=16),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=8),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighborsWeightedNewsvendor(co=5, cu=5, n_neighbors=4),
 KNeighbor